# DATASCIENTEST - SQL

## 1. Introduction 

In [ ]:
# Exécutez cette cellule pour importer les fonctions que l'on va utiliser
from IPython.display import Markdown, display

In [ ]:
#exemples d'affichage de la définition d'une table
display(Markdown("Utilisateurs ( <u>UtilisateurId</u> , Prenom, Nom , Pays de residence )"))

display(Markdown("Modules ( <u>ModuleId</u> , Nom , Difficulte )"))
display(Markdown("Utilisateurs (<u>UtilisateurId</u>, Nom, Prénom, #MetierId )")) 
display(Markdown("Metiers( <u>MetierId</u>, Nom )")) 
display(Markdown("L'attribut *MetierId* de la table **Utilisateurs** fait référence à la clé primaire de la table **Metiers** (= clé étrangère). "))

In [ ]:
#exemple clé composite
display(Markdown("Candidature(<u>#IdEtudiant</u>, Session, <u>#IdUniversite</u>, Adresse, Nom Parcours)"))
display(Markdown("* Un étudiant peut effectuer plusieurs candidatures dans une même université à des parcours différents"))
display(Markdown("* L’attribut **IdEtudiant** fait référence à la clé primaire de la table **Etudiants**"))
display(Markdown("* L’attribut **IdUniversite** fait référence à la clé primaire de la table **Universite**"))
display(Markdown("Etudiants(Nom, Prenom, Adresse, <u>IdEtudiant</u>)"))
display(Markdown("Universite (<u>IdUnversite</u>, NomUni, AdresseUniversite, Rang )"))

##### Contraintes
Les contraintes permettent de forcer les données à respecter des schémas prédéfinis. On en distingue plusieurs :

- La contrainte de domaine : Elle définit quelle valeur peut prendre un attribut, c'est à dire le type de données et des restrictions sur ce type.


- La contrainte de clé : Une clé est un groupe d'attributs qui permet d'identifier un tuple de la table. On distingue la clé primaire qui va permettre d'organiser physiquement la relation.


- La contrainte de table : C'est une contrainte qui généralise les deux précédentes. Lors de la création d'une table, on ajoutera une contrainte sur les valeurs que peuvent prendre les attributs de cette dernière.


- La contrainte d'intégrité référentielle : Un attribut ne peut prendre comme valeur qu'une valeur existante comme clé (primaire ou non) d'une autre relation. Elle interdit donc l'existence d'un tuple qui fait référence à un tuple inexistant dans une autre table.

## 2. SQL Alchemy

In [ ]:
import sqlite3, sqlalchemy
from sqlalchemy import Table, Column, Integer, String, ForeignKey, MetaData, create_engine, text, inspect
from IPython.display import Markdown, display

SQL Alchemy est une librairie Python très utilisée pour se connecter à un système de gestion de bases de données relationnelles. Il permet d'interpréter du code SQL.

On utilisera une base de données SQLite, plus légère et plus simple d'utilisation que la plupart des autres SGDBR.

Le premier objet important de ce module est l'engine. Il nous permet de nous connecter à une base de données et d'effectuer des requêtes SQL au format texte.
La fonction create_engine crée une connexion avec une base de données. Si elle n'existe pas, la fonction va en créer une.
L'argument est une chaîne de caractère qui a le format suivant : 'sqlite:///chemin_vers_la_base_de_donnees.db'

In [ ]:
engine = create_engine('sqlite:///college.db', echo=True)
# En ajoutant l'argument echo = True, cela nous permet d'afficher la traduction en langage SQL des commandes du module.

In [ ]:
#L'objet créé par la fonction MetaData permet de regrouper plusieurs informations concernant une ou plusieurs bases de données, notamment des informations sur les tables qui la composent.
meta = MetaData()

In [ ]:
#Création de table students
students = Table(
   'students', meta, 
   Column('id', Integer, primary_key=True), 
   Column('name', String), 
   Column('lastname', String),
)
meta.create_all(engine)

# Grâce au echo=True, on peut voir le code utilisé pour créer la table :

# CREATE TABLE students (
# id INTEGER NOT NULL, 
# name VARCHAR, 
# lastname VARCHAR, 
# PRIMARY KEY (id)
# )

In [ ]:
#création table parcours
parcours = Table(
   'parcours', meta, 
   Column('id_parcours', Integer, primary_key=True), 
   Column('name', String), 
   Column('student_id', Integer, ForeignKey("students.id")),
)
meta.create_all(engine)

On parle de transactions lorsqu'une succession d'opérations nécessitant des accès en lecture ou en écriture est effectuée via un système de gestion de bases de données.


Une transaction ACID est une transaction qui respecte les points suivants :
- **Atomicité** : une transaction doit être effectuée en entier ou abandonnée, c'est à dire que si une opération est refusée, toutes les opérations de la transaction ne seront pas validées.


- **Cohérente** : l'état initial et final doivent être cohérents, c'est à dire respecter les contraintes d'intégrité.


- **Isolation** : Une transaction doit s'effectuer indépendamment des transactions qui s'exécutent en même temps, c'est à dire que les états intermédiaires d'une transaction sont cachés aux autres.

- **Durabilité** : lorsqu'une transaction est validée, ses effets ne peuvent être perdus.

##### Insertion de lignes

Dans une table, on peut insérer ou supprimer des lignes. 

Ces opérations sont considérées comme des **<u>transactions<u>** et sont donc soumises aux règles qui assurent la cohérence de la base de données.


Pour insérer des tuples, on doit d'abord créer une liste qui contient des tuple. Ces tuple doivent représenter les valeurs des différents attributs de la table : la liste a donc la longueur du nombre de lignes qu'on veut insérer et chaque tuple a le même nombre d'entrées que la table.

In [ ]:
values = [(1, "Jean", "Dubois"), (2, "Florent", "Desbiolles")]

Une transaction a un début et une fin. Le début d'une transaction est indiqué par la fonction begin associée à l'objet Connection.


Via cet objet, on pourra appeller la fonction rollback dans le cas où une opération de la transaction n'a pas été autorisée : on revient à l'état initial de la table. Mais aussi la fonction commit qui permet de valider la transaction dans le cas contraire.


En Python, ça nous donne la structure suivante :

##### on crée la connection     
with engine.connect() as connection:
    # début de la transaction
    with connection.begin() as transaction:
        # on tente d'éxécuter une transaction
        try:
            connection.execute(instruction)
        # si la transaction échoue
        except:
            transaction.rollback()
            raise
        # si la transaction réussit
        else:
            transaction.commit()
La syntaxe pour ajouter un tuple dans une table est la suivante :

INSERT INTO nom_table VALUES tuple


Cette commande est en langage SQL, et donc sera au format texte sous python si on souhaite l'exécuter. La valeur doit donc être spécifiée via un paramètre associé. On note 'INSERT OR REPLACE INTO students VALUES ({markers})' où markers sera une variable explicitant le format des tuples que l'on injectera dans la table.


Une fois le markers définit, on peut mettre à jour notre requête SQL en texte en modifiant son format, c'est pour que le markers ne soit pas considéré comme une chaîne de caractères mais bien un "argument". Ensuite il suffit d'utiliser la fonction execute de l'objet Connection en passant en argument la commande texte SQL et le vecteur de valeur du paramètre associé (ici markers).

In [ ]:
# on crée la connection
with engine.connect() as connection:
    # début de la transaction
    with connection.begin() as transaction:
        # on tente d'éxécuter une transaction
        try:
            # On indique le format d'un tuple de cette table
            markers = ','.join('?' * len(values[0])) 
            
            # On utilise le langage SQL en format texte où markers est le format d'un tuple
            ins = 'INSERT OR REPLACE INTO {tablename} VALUES ({markers})'
            
            # On précise ce format particulier grâce à la fonction membre format
            ins = ins.format(tablename=students.name, markers=markers)
            
            # Enfin on peut utiliser les tuples créés en éxécutant la commande SQL
            connection.execute(ins, values)
        # si la transaction échoue
        except:
            transaction.rollback()
            raise
        # si la transaction réussit
        else:
            transaction.commit()

In [ ]:
#on  ajoute les deux parcours dans la base de données, on veillera à avoir un étudiant inscrit dans chaque parcours.


In [ ]:
values_parcours = [(1, "Data engineering", 1),(2, "Data Science", 2) ]

with engine.connect() as connection:
    with connection.begin() as transaction:
        try:
            markers = ','.join('?' * len(values_parcours[0]))
            ins = 'INSERT OR REPLACE INTO {tablename} VALUES ({markers})'
            
            ins = ins.format(tablename=parcours.name, markers=markers)
            connection.execute(ins, values_parcours)
        except:
            transaction.rollback()
            raise
        else:
            transaction.commit()

In [ ]:
# Exécuter la cellule suivante pour vérifier que vous avez bien ajouté les lignes.
with engine.connect() as connection:
    results = connection.execute("SELECT * FROM students;")
    print(results.fetchall())

with engine.connect() as connection:
    results = connection.execute("SELECT * FROM parcours;")
    print(results.fetchall())

In [ ]:
#suppression de table
sql = text('DROP TABLE IF EXISTS students;')
result = engine.execute(sql)

###Vérification que la table students n'existe plus en réutilisant le code d'insertion de tuple

#values = [
#    (1,"Jean", "Dubois"),
#    (2,"Florent", "Desbiolles"),
#    ]
#
#with engine.connect() as connection:
#    with connection.begin() as transaction:
#        try:
#            markers = ','.join('?' * len(values[0])) 
#            ins = 'INSERT INTO students VALUES ({markers})'
#            ins = ins.format(tablename=students.name, markers=markers)
#            connection.execute(ins, values)
#        except:
#            transaction.rollback()
#            raise
#        else:
#            transaction.commit()

### 3. Exemple d'application (base chinook)

sqlalchemy permet de manipuler des bases de données de types différents (Oracle, postgreSQL ...), à travers une seule et même syntaxe.

Chinook is a sample database available for SQL Server, Oracle, MySQL, etc. It can be created by running a single SQL script. Chinook database is an alternative to the Northwind database, being ideal for demos and testing ORM tools targeting single and multiple database servers.

In [ ]:
# import os
# path = str(os.getcwd())
# print(path)

In [ ]:
import sqlite3, sqlalchemy
from sqlalchemy import inspect, create_engine, text

engine = create_engine("sqlite:///chinook.db")

Une classe très pratique pour afficher des informations sur la base est l'Inspector.

In [ ]:
inspector = inspect(engine)

A l'aide de cet objet, on va pouvoir récupérer le nom des différentes tables qui constituent la base de données. La méthode get_table_names nous permet de le faire.

In [ ]:
inspector.get_table_names()

In [ ]:
print('Tables présentes dans la base de données :\n', inspect(engine).get_table_names(), '\n')
print("Attributs de la table 'albums' :\n", inspector.get_columns(table_name='albums'), '\n')
print("Foreign key(s) associée(s) à la table 'albums':\n", inspector.get_foreign_keys(table_name='albums'), '\n')

##### Pour effectuer des requêtes SQL, on a besoin d'une connexion avec l'objet engine. Pour cela on utilise la méthode connect de l'objet engine. Cette fonction renvoie une instance de la classe Connection.

In [ ]:
conn = engine.connect()
#On peut désormais utiliser la méthode execute associée à une instance de la classe Connection créée
result = conn.execute("SELECT Title FROM albums LIMIT 10;")
#Pour obtenir cette requête dans Python, on utilise la méthode fetchall associée à l'instance result de la classe ResultProxy.
result.fetchall() 
#Cette commande renvoie tous les tuples de la table de l'objet ResultProxy

##### On utilise la fonction text du module SQLAlchemy pour créer une chaîne de caractères plus adaptée


La fonction text permet de spécifier des arguments prenant des valeurs numériques dans une chaîne de caractères. Elle identifie aussi les colonnes en interne et permet aussi d'être utilisé comme sous-requêtes. On prendra l'habitude d'utiliser exclusivement cette fonction pour les requêtes SQL au format texte.

In [ ]:
#WHERE
ma_requete = text( "SELECT * FROM albums WHERE albums.ArtistId = 1")
result = conn.execute(ma_requete2)
result.fetchall() 

In [ ]:
#WHERE ... AND
#commandes qui ont un montant supérieur à 20 dollars et inf à 22 doll
stmt = text( "SELECT * FROM invoices WHERE Total > 20 AND Total < 22;")
result = conn.execute(stmt)
result.fetchall()

In [ ]:
#BETWEEN ... AND
#L’opérateur BETWEEN suivi de AND est utilisé dans une requête SQL pour sélectionner un intervalle de données dans une requête utilisant WHERE
#Afficher les identifiants, le pays dans lequel a été effectué le paiement et le montant des factures dont le montant est compris entre 10 et 20 dollars.
stmt = text( "SELECT InvoiceId, BillingCountry, Total FROM invoices WHERE Total BETWEEN 10 AND 20;")
result = conn.execute(stmt)
result.fetchall()

##### Expressions régulières


| Expression régulière | Decription |
| --- | --- |
| .	| correspond à n'importe quel caractère seul |
| *	| correspond à n'importe quel caractère |
| +	| correspond à au moins une instance de l'expression précédente |
| ^	| correspond à un commencement en début de ligne |
| $	| recherche à la fin de la ligne |
| <	| correspondance uniquement si le mot commence en ce point |
| >	| correspondance uniquement si le mot finit en ce point |
| \n | correspondance si il y a un retour à la ligne |
| [ ] | correspondance si il y a présence de n'importe quel des caractères entre crochets |
| [^...] |  correspondance avec n'importe quel caractère listé après ^ |
| [ABQ]% | la chaîne de caractères doit commencer par un A , B ou Q et peut être de n'importe quelle longueur |
| [AB][CD]% | la chaîne de caractères doit commencer par un A ou B et le deuxième caractère doit être C ou D |
| [A-Z]% | la chaîne doit commencer par un caractère de A à Z et peut être de n'importe quelle longueur |
| [A-Z0-9]% | la chaîne doit commencer par un lettre de A à Z ou un chiffre de 0 à 9 et peut être de n'importe quelle longueur |
| [^A-C]% | la chaîne ne peut pas commencer par un caractère de A à C mais peut être de n'importe quelle longueur |
| %[A-Z] | la chaîne de caractère doit se terminer par un caractère de A à Z et peut être de n'importe quelle longueur |
| %[%\$#@]% | la chaine de caractères peut être de n'importe quelle longueur et doit au moins avoir une correspondance avec un caractère entre crochets |

On présente ici la façon la plus flexible d'utiliser des expressions régulières avec SQL, à savoir l'utilisation du mot clé LIKE et des Wildcards.

On distingue 2 types de caractères spéciaux comme Wildcards :

- le % désigne n'importe quelle combinaison de caractère de n'importe quelle longueur.
- le _ désigne n'importe quel caractère unique.

Voici un exemple de modèle : P% qui aura une correspondance avec n'importe quelle chaîne de caractères qui commence par un P. On peut aussi utiliser ce modèle %P% qui aura une correspondance avec n'importe quelle chaîne de caractères qui contient un P. Dans une requête SQL, cela donne :


**SELECT * FROM table1 WHERE colonne1 LIKE '%P';**


In [ ]:
#WHERE ... LIKE
#Chercher les tuples dans la table tracks dont le titre de la musique contient Wild
stmt = text ( "SELECT * FROM tracks WHERE Name LIKE '%Wild%';")
result = conn.execute(stmt)
result.fetchall()

In [ ]:
#WHERE ... LIKE ... AND
#Afficher les musiques composées par Freddie Mercury ou en collaboration avec d'autres artistes.
stmt = text ( "SELECT tracks.Name, tracks.Composer FROM tracks WHERE Composer LIKE '%Freddie%' AND Composer LIKE '%Mercury%'" )
result = conn.execute(stmt)
result.fetchall()

In [ ]:
#WHERE ... LIKE ... AND (LIKE ... AND ... NOT LIKE)
#Afficher les musiques dont le titre commence par un P et qui sont issues d'une association d'artistes.
#Indice : Pour une association d'artistes, le nom est séparé par un / ou - à l'exception du groupe AC/DC.
stmt = text ( "SELECT tracks.Name, tracks.Composer FROM tracks"
             " WHERE Name LIKE 'P%' AND ((Composer LIKE '%/%' AND Composer NOT LIKE '__/__')OR Composer LIKE '%-%') ")
result = conn.execute(stmt)
result.fetchall()

In [ ]:
#ORDER BY
# On peut simplement ajouter les noms des attributs à extraire les uns à la suite des autres séparés par une virgule
stmt = text ( "SELECT InvoiceId, CustomerId, Total FROM invoices ORDER BY Total DESC LIMIT 10;")
result = conn.execute(stmt)
result.fetchall()

In [ ]:
#GROUP BY
#montant total (SUM) des commandes par identifiant client CustomerId).
# on a aussi choisi de classer les résultats par montant total décroissant
stmt = text ( "SELECT invoices.CustomerId , SUM(Total) AS tot FROM invoices GROUP BY CustomerId ORDER BY tot DESC;")
result = conn.execute(stmt)
result.fetchall()

In [ ]:
#AVG
#AS
#Le mot clé AS est utilisé pour renommer une table ou une colonne de table. C'est ce qu'on appelle l'Alias.
#moyenne de prix des commandes par pays, et renommer la nouvelle colonne par PaysMoy
stmt = text ( "SELECT BillingCountry, AVG(Total) AS PaysMoy FROM invoices GROUP BY BillingCountry " )
result = conn.execute(stmt)
result.fetchall()

In [ ]:
#MAX
#commande la plus chère effectuée par pays et ranger ces valeurs par ordre décroissant.
stmt = text ( "SELECT BillingCountry, MAX(Total) AS tot FROM invoices GROUP BY BillingCountry ORDER BY tot DESC " )
result = conn.execute(stmt)
result.fetchall()

In [ ]:
#COUNT(*)
#nombre de commandes effectuées par pays.
stmt = text ( "SELECT BillingCountry, COUNT(*) AS p FROM invoices GROUP BY BillingCountry ORDER BY p")
result = conn.execute(stmt)
result.fetchall()

In [ ]:
#clés d'aggrégation multiples
#nombre de commandes effectuées par pays et par ville:
# Chaque paire (PAYS, VILLE) existant dans la base est représentée.
stmt = text ( "SELECT BillingCountry, BillingCity, COUNT(InvoiceId)"
             " FROM invoices GROUP BY BillingCountry, BillingCity" )
result = conn.execute(stmt)
result.fetchall()

In [ ]:
#HAVING (toujours précédée d'un GROUP BY)
# Cette clause fonctionne exactement comme la clause WHERE mais pour des fonctions d'agrégats. 
#Elle permet de spécifier un prédicat sur le résultat d'un GROUP BY, elle est donc forcément précédée par cette dernière clause.
# Si on utilise la clause WHERE, on élimine des lignes avant l'agrégation.
#identifiant des clients qui ont un nombre de commandes supérieur à 5. Ranger ces valeurs par ordre croissant.
stmt = text ("SELECT CustomerId, COUNT(CustomerId) AS tot FROM invoices GROUP BY CustomerId "
             "HAVING tot > 5 ORDER BY tot" )
result = conn.execute(stmt)
result.fetchall()

In [ ]:
#HAVING (toujours précédée d'un GROUP BY)
#liste des clients dont le montant total des commandes est supérieur à 20 dollars.
stmt = text ( "SELECT CustomerId, SUM(Total) AS tot FROM invoices GROUP BY CustomerId HAVING tot > 20;" )
result = conn.execute(stmt)
result.fetchall()

In [ ]:
#HAVING (toujours précédée d'un GROUP BY)
# Le WHERE agit par ligne, le HAVING agit par aggrégat
#liste des clients et le montant total de leurs commandes dont le montant est supérieur à 20 dollars.
stmt = text ( "SELECT CustomerId, SUM(Total) AS tot FROM invoices WHERE Total > 20 GROUP BY CustomerId " )
result = conn.execute(stmt)
result.fetchall()

### 4. SQL Avancé

##### Requêtes imbriquées

In [ ]:
'''
La première requête récupère les IDs des 10 clients avec les plus gros montants.
La deuxième requête classe ces résultats par ordre croissant.
'''
stmt = text("SELECT * FROM " #on créé ensuite la table temporare via une requete sql entre ()
            "(SELECT CustomerId, SUM(Total) AS total FROM invoices "
            "GROUP BY CustomerId ORDER BY Total DESC LIMIT 10) table_temp "
            "ORDER BY Total")
result = conn.execute(stmt)
result.fetchall()
#Dans cet exemple, on a une première requête entre parenthèses. 
#Cette requête est suivie du nom que l'on donne à la table temporaire table_temp
#La structure de la requête est ici :
#SELECT * FROM (SELECT * FROM table1) nom_de_la_table_temporaire;

##### Jointures

In [ ]:
#Produit cartésien ou CROSS JOIN
stmt = text( "SELECT * FROM tracks CROSS JOIN genres LIMIT 30")
result = conn.execute(stmt)
result.fetchall()

In [ ]:
#Jointure interne ou INNER JOIN
#Ce type de jointure va permettre de concaténer les tuples des 2 tables deux à deux si une condition est satisfaite.
stmt = text ( "SELECT * FROM artists INNER JOIN albums ON artists.ArtistId = albums.ArtistId LIMIT 10;" )
result = conn.execute(stmt)
result.fetchall()

In [ ]:
#Jointure à droite ou RIGHT JOIN (mmem chose en sens inverse pour LEFT JOIN)
#La jointure à droite n'est pas supporté par le module SQLAlchemy
#La jointure à gauche ou LEFT JOIN est une jointure entre 2 tables qui permet de retourner tous les 
#enregistrements de la table de gauche même s’il n’y a pas de correspondance avec la table de droitee
stmt =text ("SELECT * FROM (SELECT CustomerId, Total  FROM 'invoices' WHERE InvoiceId BETWEEN 21 AND 40) AS A "
            "LEFT JOIN (SELECT CustomerId, Total FROM 'invoices' "
            "WHERE InvoiceId BETWEEN 1 AND 20) AS B ON A.CustomerId=B.CustomerId")
result = conn.execute(stmt)
result.fetchall()

##### Modifier une table

Jusqu'à présent, nous avons vu comment faire des requêtes pour récupérer des données. Mais on peut vouloir modifier ou supprimer des lignes. 

En connaissant comment récupérer des données, on peut faire des modifications précises. On ne proposera pas d'exercice de modification dans ce cours pour ne pas risquer de casser la base de données mais vous pouvez essayer de votre côté, à vos risques et périls.

###### Modifier une ligne avec SET et UPDATE
On peut utiliser UPDATE et SET pour modifier une ou plusieurs ligne(s).
Par exemple, si on veut modifier la colonne colonne1 pour qu'elle prenne la valeur nouvelle_valeur lorsque la colonne colonne2 vaut valeur, on peut écrire :

UPDATE table1
SET colonne1 = nouvelle_valeur WHERE colonne2 = valeur;


###### Supprimer des lignes avec DELETE
Sur le même principe, on peut supprimer les lignes pour lesquelles la colonne colonne2 vaut valeur en faisant :

DELETE FROM table WHERE colonne2 = valeur;

In [ ]:
======== Tables présentes dans la base de données =========
'crew', 'episodes', 'people', 'ratings', 'titles'

====== Attributs de la table crew :  ======
'title_id', 'person_id', 'category', 'job', 'characters'

====== Attributs de la table episodes :  ======
'episode_title_id', 'show_title_id', 'season_number', 'eposide_number' 

====== Attributs de la table people :  ======
'person_id', 'name', 'born', 'died'

====== Attributs de la table ratings :  ======
'title_id', 'rating', 'votes'

====== Attributs de la table titles :  ======
'title_id', 'type', 'primary_title', 'original_title', 'is_adult', 
'premiered', 'ended', 'runtime_minutes', 'genres'


# Examen

In [1]:
import sqlite3, sqlalchemy
from sqlalchemy import Table, Column, Integer, String, Float, ForeignKey, MetaData, create_engine, text, inspect
from IPython.display import Markdown, display

In [2]:
engine = create_engine('sqlite:///Distributeur.db', echo=True)
meta = MetaData()

#Création de table Boisson
Boissons = Table(
   'Boissons', meta, 
   Column('BoissonId', Integer, primary_key=True), 
   Column('Nom', String), 
   Column('Contenance', Integer),
    Column('Prix', Float),
)
meta.create_all(engine)


2021-08-25 06:09:02,152 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-08-25 06:09:02,159 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 06:09:02,168 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-08-25 06:09:02,172 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 06:09:02,178 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Boissons")
2021-08-25 06:09:02,181 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 06:09:02,189 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Boissons")
2021-08-25 06:09:02,193 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 06:09:02,201 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Boissons" (
	"BoissonId" INTEGER NOT NULL, 
	"Nom" VARCHAR, 
	"Contenance" INTEGER, 
	"Prix" FLOAT, 
	PRIMARY KEY ("BoissonId")
)


2021-08-25 06:09:02,202 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 06:09:02,215 INFO sqlalchemy.engine.base.Engine

In [3]:
values = [(1, 'Coca', 33, 0.99), (2, 'Perrier', 100, 2), (3, 'Perrier', 33, 1), 
          (4, 'Vittel', 150, 1.50), (5, 'Badoit', 50, 2)]

# on crée la connection
with engine.connect() as connection:
    # début de la transaction
    with connection.begin() as transaction:
        # on tente d'éxécuter une transaction
        try:
            # On indique le format d'un tuple de cette table
            markers = ','.join('?' * len(values[0])) 
            
            # On utilise le langage SQL en format texte où markers est le format d'un tuple
            ins = 'INSERT OR REPLACE INTO {tablename} VALUES ({markers})'
            
            # On précise ce format particulier grâce à la fonction membre format
            ins = ins.format(tablename=Boissons.name, markers=markers)
            
            # Enfin on peut utiliser les tuples créés en éxécutant la commande SQL
            connection.execute(ins, values)
        # si la transaction échoue
        except:
            transaction.rollback()
            raise
        # si la transaction réussit
        else:
            transaction.commit()

2021-08-25 06:09:02,884 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-08-25 06:09:02,892 INFO sqlalchemy.engine.base.Engine INSERT OR REPLACE INTO Boissons VALUES (?,?,?,?)
2021-08-25 06:09:02,895 INFO sqlalchemy.engine.base.Engine [(1, 'Coca', 33, 0.99), (2, 'Perrier', 100, 2), (3, 'Perrier', 33, 1), (4, 'Vittel', 150, 1.5), (5, 'Badoit', 50, 2)]
2021-08-25 06:09:02,902 INFO sqlalchemy.engine.base.Engine COMMIT


In [4]:
#Création de table Facture
Facture = Table(
   'Facture', meta, 
   Column('FactureId', Integer, primary_key=True), 
   Column('BoissId', Integer, ForeignKey("Boissons.BoissonId")), 
   Column('Heure', Integer),
)
meta.create_all(engine)

2021-08-25 06:09:03,636 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Boissons")
2021-08-25 06:09:03,637 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 06:09:03,640 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Facture")
2021-08-25 06:09:03,642 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 06:09:03,645 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Facture")
2021-08-25 06:09:03,647 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 06:09:03,652 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Facture" (
	"FactureId" INTEGER NOT NULL, 
	"BoissId" INTEGER, 
	"Heure" INTEGER, 
	PRIMARY KEY ("FactureId"), 
	FOREIGN KEY("BoissId") REFERENCES "Boissons" ("BoissonId")
)


2021-08-25 06:09:03,654 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 06:09:03,662 INFO sqlalchemy.engine.base.Engine COMMIT


In [5]:
values = [(1, 1, 7), (2, 1, 8), (3, 4, 12)]

# on crée la connection
with engine.connect() as connection:
    # début de la transaction
    with connection.begin() as transaction:
        # on tente d'éxécuter une transaction
        try:
            # On indique le format d'un tuple de cette table
            markers = ','.join('?' * len(values[0])) 
            
            # On utilise le langage SQL en format texte où markers est le format d'un tuple
            ins = 'INSERT OR REPLACE INTO {tablename} VALUES ({markers})'
            
            # On précise ce format particulier grâce à la fonction membre format
            ins = ins.format(tablename=Facture.name, markers=markers)
            
            # Enfin on peut utiliser les tuples créés en éxécutant la commande SQL
            connection.execute(ins, values)
        # si la transaction échoue
        except:
            transaction.rollback()
            raise
        # si la transaction réussit
        else:
            transaction.commit()

2021-08-25 06:09:04,385 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-08-25 06:09:04,388 INFO sqlalchemy.engine.base.Engine INSERT OR REPLACE INTO Facture VALUES (?,?,?)
2021-08-25 06:09:04,390 INFO sqlalchemy.engine.base.Engine [(1, 1, 7), (2, 1, 8), (3, 4, 12)]
2021-08-25 06:09:04,397 INFO sqlalchemy.engine.base.Engine COMMIT


In [6]:
'''On ne peut pas supprimer la table Boissons tant que l'argument Boisson.BoissonId est présent dans la table Facture en tant que clé étrangère'''

"On ne peut pas supprimer la table Boissons tant que l'argument Boisson.BoissonId est présent dans la table Facture en tant que clé étrangère"

In [7]:
'''On ne peut pas rajouter ce tuple puisque l'argument BoissonId est clé primaire de la table Boissons, et la valeur BoissonId=1 existe déjà. Or on sait que les valeurs des clés primaires doivent être uniques'''

"On ne peut pas rajouter ce tuple puisque l'argument BoissonId est clé primaire de la table Boissons, et la valeur BoissonId=1 existe déjà. Or on sait que les valeurs des clés primaires doivent être uniques"

In [8]:
#quelles boissons du ditrib n'ont pas été vendues ?
conn = engine.connect()

stmt = text("SELECT BoissonId, Nom FROM Boissons "
            "WHERE BoissonId NOT IN (SELECT BoissId From Facture)")
result = conn.execute(stmt)
result.fetchall() 

2021-08-25 06:09:06,387 INFO sqlalchemy.engine.base.Engine SELECT BoissonId, Nom FROM Boissons WHERE BoissonId NOT IN (SELECT BoissId From Facture)
2021-08-25 06:09:06,390 INFO sqlalchemy.engine.base.Engine ()


[(2, 'Perrier'), (3, 'Perrier'), (5, 'Badoit')]

In [9]:
#quelle proportion des différentes boissons vendues ?
stmt = text ( "SELECT BoissonId, Nom, COUNT(*) * 100.0 / SUM(COUNT(*)) over()"
             "FROM Facture LEFT JOIN Boissons "
             "ON Facture.BoissId = Boissons.BoissonId GROUP BY BoissonId" )
result = conn.execute(stmt)
result.fetchall() 



2021-08-25 06:09:07,282 INFO sqlalchemy.engine.base.Engine SELECT BoissonId, Nom, COUNT(*) * 100.0 / SUM(COUNT(*)) over()FROM Facture LEFT JOIN Boissons ON Facture.BoissId = Boissons.BoissonId GROUP BY BoissonId
2021-08-25 06:09:07,284 INFO sqlalchemy.engine.base.Engine ()


[(1, 'Coca', 66.66666666666667), (4, 'Vittel', 33.333333333333336)]

In [10]:
#supprimer les 2 tables
sql = text('DROP TABLE IF EXISTS Boissons;')
result = engine.execute(sql)

sql = text('DROP TABLE IF EXISTS Facture;')
result = engine.execute(sql)

2021-08-25 06:09:08,180 INFO sqlalchemy.engine.base.Engine DROP TABLE IF EXISTS Boissons;
2021-08-25 06:09:08,188 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 06:09:08,197 INFO sqlalchemy.engine.base.Engine COMMIT
2021-08-25 06:09:08,201 INFO sqlalchemy.engine.base.Engine DROP TABLE IF EXISTS Facture;
2021-08-25 06:09:08,204 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 06:09:08,212 INFO sqlalchemy.engine.base.Engine COMMIT


In [11]:
def tables_scheme():
    print_list = []
    for t in meta.sorted_tables:
        list_col_PK = []
        list_col_FK = []
        list_col = []
        for c in t.columns:
            if c.primary_key==True:
                col = c.key
                list_col_PK.append(col)
            elif c.foreign_keys:
                col = str('#')+c.key
                list_col_FK.append(col)
            else:
                col = c.key
                list_col.append(col)
        list_col_FK.extend(list_col)
        print_list.append(Markdown("{} (<u>{}</u>, {})".format(t, ", ".join(list_col_PK),
                                                               ", ".join(list_col_FK))))
    return print_list

In [12]:
print(engine)
for table_scheme in tables_scheme():
    display(table_scheme)

Engine(sqlite:///Distributeur.db)


Boissons (<u>BoissonId</u>, Nom, Contenance, Prix)

Facture (<u>FactureId</u>, #BoissId, Heure)

### Exercice 2

In [13]:
#a) connecxion
engine = create_engine('sqlite:///newIMDB.db', echo=True)
meta = MetaData()

2021-08-25 06:09:12,438 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-08-25 06:09:12,440 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 06:09:12,443 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-08-25 06:09:12,444 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 06:09:12,445 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Boissons")
2021-08-25 06:09:12,447 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 06:09:12,450 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Boissons")
2021-08-25 06:09:12,454 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 06:09:12,457 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Boissons" (
	"BoissonId" INTEGER NOT NULL, 
	"Nom" VARCHAR, 
	"Contenance" INTEGER, 
	"Prix" FLOAT, 
	PRIMARY KEY ("BoissonId")
)


2021-08-25 06:09:12,457 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 06:09:12,466 INFO sqlalchemy.engine.base.Engine

In [14]:
def tables_scheme():
    print_list = []
    for t in meta.sorted_tables:
        list_col_PK = []
        list_col_FK = []
        list_col = []
        for c in t.columns:
            if c.primary_key==True:
                col = c.key
                list_col_PK.append(col)
            elif c.foreign_keys:
                col = str('#')+c.key
                list_col_FK.append(col)
            else:
                col = c.key
                list_col.append(col)
        list_col_FK.extend(list_col)
        print_list.append(Markdown("{} (<u>{}</u>, {})".format(t, ", ".join(list_col_PK),
                                                               ", ".join(list_col_FK))))
    return print_list

print(engine)
for table_scheme in tables_scheme():
    display(table_scheme)

Engine(sqlite:///newIMDB.db)


Boissons (<u>BoissonId</u>, Nom, Contenance, Prix)

In [15]:
======== Tables présentes dans la base de données =========
'crew', 'episodes', 'people', 'ratings', 'titles'



====== Attributs de la table crew :  ======
'title_id', 'person_id', 'category', 'job', 'characters'

====== Attributs de la table episodes :  ======
'episode_title_id', 'show_title_id', 'season_number', 'eposide_number' 

====== Attributs de la table people :  ======
'person_id', 'name', 'born', 'died'

====== Attributs de la table ratings :  ======
'title_id', 'rating', 'votes'

====== Attributs de la table titles :  ======
'title_id', 'type', 'primary_title', 'original_title', 'is_adult', 
'premiered', 'ended', 'runtime_minutes', 'genres'

SyntaxError: invalid syntax (<ipython-input-15-845c5d811fc4>, line 1)

In [18]:
#c) OK
conn = engine.connect()

#titres les mieux notés. Indiquer nom, note, nbre de votants (avec nb votants > 5000). 
#limiter nb de résultats à 10
stmt = text(
    "SELECT original_title, rating, votes "
    "FROM titles "
    "LEFT JOIN ratings "
    "ON titles.title_id=ratings.title_id "
    "WHERE ratings.votes > 5000 "
    "ORDER BY votes DESC LIMIT 10 "
)
result = conn.execute(stmt)
result.fetchall()  

In [ ]:
#d) OK
#note moyenne des titres par catégorie, classer ce résultat par ordre décroissant des notes moyennes ? (On limitera les résultats aux 10 premières lignes.)
# Insérez votre code ici
stmt = text(
    "SELECT type, AVG(ratings.rating) FROM ratings "
    "LEFT JOIN titles ON ratings.title_id=titles.title_id "
    "GROUP BY type "
    "ORDER BY votes DESC LIMIT 10 "
)
result = conn.execute(stmt)
result.fetchall() 

In [ ]:
#e) notes moyennes des titres associés aux différents 
#directeurs/metteurs en scène/réalisateur (on limite aux 10 premières lignes)
stmt = text(
    "SELECT name, AVG(rating) as NoteMoy FROM crew "
    "LEFT JOIN ratings ON crew.title_id=ratings.title_id "
    "WHERE category = 'director' AS A "
    "LEFT JOIN people ON A.person_id=people.person_id as B "
    "GROUP BY B.person_id" #(à vérifier)
    "ORDER BY NoteMoy DESC LIMIT 10"
)
result = conn.execute(stmt)
result.fetchall() 

In [17]:
#f) OK
#combien y-a-t-il de titres par type ?
stmt = text(
    "SELECT original_title, type, COUNT(title_id) FROM titles "
    "GROUP BY type " 
)
result = conn.execute(stmt)
result.fetchall() 

In [ ]:
#g) OK
#quels sont les titres sur lesquels Quentin Tarantino a travaillé ?
#on précisera le nom et la note du titre
stmt = text( "SELECT primary_title, rating FROM ((titles "
             "LEFT JOIN ratings ON titles.title_id=ratings.title_id) AS A "
             "LEFT JOIN crew ON A.title_id=crew.title_id) AS B "
             "LEFT JOIN people ON B.person_id=people.person_id "
             "WHERE people.name LIKE '%Tarantino%' " 
            )
result = conn.execute(stmt)
result.fetchall()

In [ ]:
#OK h) quels sont les acteurs ou actrices ayant obtenu les meilleurres note (>9)?
# la variable category de la variable crew détermine le métier de l'individu auquel le tupel est associé.
#limite aux 10 premiers résultats
stmt = text( "SELECT name, rating FROM ((SELECT * FROM crew "
            "LEFT JOIN ratings ON crew.title_id=ratings.title_id "
            "WHERE rating > 9 "
            "AND category = 'actor') AS A "
            "LEFT JOIN people ON A.person_id=people.person_id) AS B "
#             "GROUP BY B.person_id "
            "ORDER BY rating DESC "
            "LIMIT 10 "
            )
result = conn.execute(stmt)
result.fetchall()

In [ ]:
#OK i) quelle est la note moyenne des contenus pour les actrices et les acteurs ?
#regrouper ces 2 infos ds un seul tuple.
stmt = text( "SELECT name, AVG(rating) as NoteMoy FROM ((SELECT * FROM crew "
            "LEFT JOIN ratings ON crew.title_id=ratings.title_id "
            "WHERE category='actor') AS A "
            "LEFT JOIN people ON A.person_id=people.person_id) AS B "
            "GROUP BY B.person_id "
            "ORDER BY NoteMoy DESC "
            )
result = conn.execute(stmt)
result.fetchall()


### 3. Question bonus
(a) Quels sont les directeurs/metteurs en scène/réalisateurs (si ils existent) qui ont obtenu une note supérieur à 9 pour tous leurs titres ? (On limitera les résultats aux 10 premières lignes.)

In [ ]:
stmt = text( "SELECT name, AVG(rating) FROM ((SELECT * FROM crew "
            "LEFT JOIN ratings ON crew.title_id=ratings.title_id "
            "AND category = 'actor') AS A "
            "LEFT JOIN people ON A.person_id=people.person_id) AS B "
            "GROUP BY B.person_id "
            "HAVING min(rating) > 9 "
            "ORDER BY rating DESC "
            "LIMIT 10 "
            )
result = conn.execute(stmt)
result.fetchall()

In [ ]:
======== Tables présentes dans la base de données =========
'crew', 'episodes', 'people', 'ratings', 'titles'


====== Attributs de la table crew :  ======
'title_id', 'person_id', 'category', 'job', 'characters'


====== Attributs de la table episodes :  ======
'episode_title_id', 'show_title_id', 'season_number', 'eposide_number' 

====== Attributs de la table people :  ======
'person_id', 'name', 'born', 'died'

====== Attributs de la table ratings :  ======
'title_id', 'rating', 'votes'

====== Attributs de la table titles :  ======
'title_id', 'type', 'primary_title', 'original_title', 'is_adult', 
'premiered', 'ended', 'runtime_minutes', 'genres'
